## Construcción de dataset de la serie del tiofen

Para construir el dataset de esta serie se usaran los compuestos con ids del 26 al 42, el smile que le corresponde, la media de su ic50 y la desviación estándar de la misma.

Primeramente se construye el dataset con los compuestos del 26 al 37 ya que mediante los LLM son fáciles de detectar y obtener su SMILE, esto luego es validado mediatne rdkit. En el caso de los compuestos que se ven como grafos se dibujan usando el HyperChem (están en carpeta mol2_files), y luego se transforman a smile con rdkit. Al final se juntan todos los compuestos en un mismo dataset con los datos obtenidos de la tabla.

In [44]:
import pandas as pd # construcción del dataset
from rdkit import Chem # obtención de smile a partir del mol2
import os # trabajo con ficheros

### Construcción de dataset inicial

In [45]:
thiophene_data = [
    {"id": 26, "SMILES": "CC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 27, "SMILES": "CCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 28, "SMILES": "C1CC1C2=C(SC=C2[N+](=O)[O-])"},
    {"id": 29, "SMILES": "OCCCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 30, "SMILES": "COCCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 31, "SMILES": "COC(CO)CC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 32, "SMILES": "COCCCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 33, "SMILES": "CCN(CC)CCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 34, "SMILES": "CN(C)CCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 35, "SMILES": "OCCCOCCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 36, "SMILES": "C=CCC1=C(SC=C1[N+](=O)[O-])"},
    {"id": 37, "SMILES": "C#CCC1=C(SC=C1[N+](=O)[O-])"},
]
df = pd.DataFrame(thiophene_data)
df.head()

,id,SMILES
0,26,CC1=C(SC=C1[N+](=O)[O-])
1,27,CCC1=C(SC=C1[N+](=O)[O-])
2,28,C1CC1C2=C(SC=C2[N+](=O)[O-])
3,29,OCCCC1=C(SC=C1[N+](=O)[O-])
4,30,COCCC1=C(SC=C1[N+](=O)[O-])


In [46]:
# Función para validar y canonicalizar
def validate_and_canonicalize(smiles):
    if not isinstance(smiles, str) or smiles.strip() == "":
        print ("empty")
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print ("invalid")
    canonical = Chem.MolToSmiles(mol, canonical=True)
    print ("valid")

# Mostrar resultados
print("=== Resultados de validación ===")
results = df["SMILES"].apply(lambda s: validate_and_canonicalize(s))


=== Resultados de validación ===
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid


### Obtención de compuestos representados mediante grafos

In [47]:
mol2_dir = "mol2_files"
new_rows = []

for filename in os.listdir(mol2_dir):
    if not filename.endswith(".ml2"):
        continue

    try:
        comp_id = int(filename.split(".")[0])
    except ValueError:
        print(f"⚠️  Nombre no numérico ignorado: {filename}")
        continue

    # Leer mol2
    mol2_path = os.path.join(mol2_dir, filename)
    mol = Chem.MolFromMol2File(mol2_path, sanitize=True, removeHs=False)

    if mol is None:
        print(f"❌ Fallo al leer {filename}")
        continue

    # Canonicalizar SMILES
    try:
        smiles = Chem.MolToSmiles(mol, canonical=True)
    except Exception as e:
        print(f"⚠️  Error al canonicalizar {comp_id}: {e}")
        continue

    # Crear nueva fila
    new_row = {
        "id": comp_id,
        "SMILES": smiles,
    }
    new_rows.append(new_row)

# Convertir a DataFrame
if new_rows:
    df_new = pd.DataFrame(new_rows)
    # Asegurar que las columnas coincidan con `df`
    df_new = df_new[df.columns]  # reordena según el orden de `df`
    df = pd.concat([df, df_new], ignore_index=True)
    print(f"✅ Añadidas {len(new_rows)} nuevas filas desde .mol2")
else:
    print("ℹ️  No se añadieron nuevas filas.")

✅ Añadidas 5 nuevas filas desde .mol2


In [48]:
df.head()

,id,SMILES
0,26,CC1=C(SC=C1[N+](=O)[O-])
1,27,CCC1=C(SC=C1[N+](=O)[O-])
2,28,C1CC1C2=C(SC=C2[N+](=O)[O-])
3,29,OCCCC1=C(SC=C1[N+](=O)[O-])
4,30,COCCC1=C(SC=C1[N+](=O)[O-])


### Unión de ambos conjuntos de datos del tiofen

In [49]:
# IDs de la serie tiofeno (26–42, incluyendo 33 y 34)
thiophene_ids = [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
# IC50_mean (media) → comas decimales corregidas a puntos
IC50_mean = [64.0, 98.4, 97.3, 3.0, 104.0, 56.0, 3.0, 36.0, 25.0, 42.0, 34.3, 61.2, 35.8, 115.6, 198.0, 228.0, 156.0]
# IC50_std (desviación estándar)
IC50_std = [0.32, 0.12, 0.98, 0.41, 0.7, 0.74, 0.5, 0.24, 0.17, 0.45, 0.5, 0.22, 0.62, 0.63, 0.48, 0.9, 0.16]

In [50]:
# Crear un DataFrame con los valores correctos
df_ic50 = pd.DataFrame({
    'id': thiophene_ids,
    'IC50_mean': IC50_mean,
    'IC50_std': IC50_std
})

# Eliminar columna antigua si existe
if 'IC50' in df.columns:
    df = df.drop(columns=['IC50'])

# Unir por 'id'
df = df.merge(df_ic50, on='id', how='left')

In [51]:
df.head()

,id,SMILES,IC50_mean,IC50_std
0,26,CC1=C(SC=C1[N+](=O)[O-]),64.0,0.32
1,27,CCC1=C(SC=C1[N+](=O)[O-]),98.4,0.12
2,28,C1CC1C2=C(SC=C2[N+](=O)[O-]),97.3,0.98
3,29,OCCCC1=C(SC=C1[N+](=O)[O-]),3.0,0.41
4,30,COCCC1=C(SC=C1[N+](=O)[O-]),104.0,0.70


In [52]:
# exportar
df.to_csv("dataset_qsar_tiofeno.csv", index=False)